# Module Import

In [1]:
# Crawling
from selenium import webdriver
import time
import pandas as pd
from bs4 import BeautifulSoup
import datetime
from datetime import timedelta

# Basic
import pandas as pd
import numpy as np
import openpyxl

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Language Compatibility
from matplotlib import font_manager, rc

font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

# Define Crawling Function

In [ ]:
def yanolja_crawl(url):
    global df # 전역 변수를 지역 범위에서 적용
    driver.get(url)

    hotel = driver.find_element_by_css_selector('section.PlaceDetailTitle_titleContainer__3sGdf h1').text
    hotel = hotel.replace("[★숙박대전] ", "")

    # 리뷰 페이지 클릭 (Click review page)
    driver.find_element_by_css_selector('section.PlaceDetailTitle_titleContainer__3sGdf > a').click()

    # 무한 스크롤
    import time
    time.sleep(1)

    SCROLL_PAUSE_TIME = 1.5

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break

        last_height = new_height

    # 스크롤 끝나고 페이지 elements 가져오기
    raw = driver.page_source # 페이지의 elements 모두 가져오기
    html = BeautifulSoup(raw, "html.parser")

    # 리뷰 텍스트 선택자: div.item-wrap
    reviews = html.select("div.item-wrap")

    columns = ['hotel', 'star', 'review', 'date']

    for r in reviews:
        review = r.select_one("p")
        date = r.select_one('time')

        stars = r.select('.container.score')
        for s in stars:
            star_count = 5 - len(s.select(".ico.gray"))

        df = df.append(pd.DataFrame([[hotel, star_count, review.text, date.text]], columns = columns),
                       ignore_index=True)
    
def change_date(x):
    now = datetime.datetime.today()
    today = now.strftime("%Y. %m. %d")
    yesterday = (now - timedelta(days=1)).strftime("%Y. %m. %d")
    _2days_ago = (now - timedelta(days = 2)).strftime("%Y. %m. %d")
    _3days_ago = (now - timedelta(days = 3)).strftime("%Y. %m. %d")
    hour = int(datetime.datetime.today().strftime("%H"))
    if "시간 전" in x:
        x = int(x.replace("시간 전", ""))
        if hour - x > 0: return today
        else: return yesterday
    elif "일 전" in x:
        if "1" in x: return yesterday
        elif "2" in x: return _2days_ago
        else: return _3days_ago
    else: return x

# Crawling

In [ ]:
url_list = [ "https://www.yanolja.com/hotel/3001317", # 라까사 호텔 서울
             "https://www.yanolja.com/guest-house/10040389", # 라까사 호텔 서울_언바운디 -> 390~393
             "https://www.yanolja.com/hotel/3016071", # 라까사 호텔 광명
             "https://www.yanolja.com/guest-house/10041118", # 라까사 호텔 광명_언바운디 -> 1386~1390
             "https://www.yanolja.com/hotel/3000613", # 코리아나 호텔
             "https://www.yanolja.com/guest-house/10040099", # 호텔 코리아나_언바운디 -> 1877~1878 / 호텔 코리아나
             "https://www.yanolja.com/hotel/3009495", # 홍대 호텔 나인브릭
             "https://www.yanolja.com/guest-house/10040690", # 호텔 나인브릭_언바운디 -> 3784 / 호텔 나인브릭
             "https://www.yanolja.com/hotel/3000720", # 더 리버사이드 호텔
             "https://www.yanolja.com/guest-house/10040173", # 호텔 더 리버사이드_언바운디 -> 4525~4527 / 호텔 더 리버사이드
             "https://www.yanolja.com/guest-house/1000108371", # 평창 고려궁 한옥호텔_언바운디 -> 4528~4537
             "https://www.yanolja.com/hotel/10040002", # 부산 스튜디오202
             "https://www.yanolja.com/guest-house/1000113591", # 부산 스튜디오202_언바운디 -> 4545~4586 / 광안리 스튜디오 202
             "https://www.yanolja.com/guest-house/1000111730" # 서귀포 일레인 호텔 _언바운디 -> 4587~4628
           ]

columns = ['hotel', 'star', 'review', 'date']
df = pd.DataFrame(columns = columns) # 데이터프레임 생성

In [ ]:
path = 'C:/Users/zyunhyeok/Desktop/자료/chromedriver/chromedriver.exe'
driver = webdriver.Chrome(path)

for url in url_list:
    yanolja_crawl(url)

# driver.close()

C:\Users\zyunhyeok\AppData\Local\Temp\ipykernel_23856\64086834.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)
C:\Users\zyunhyeok\AppData\Local\Temp\ipykernel_23856\774128163.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  hotel = driver.find_element_by_css_selector('section.PlaceDetailTitle_titleContainer__3sGdf h1').text
C:\Users\zyunhyeok\AppData\Local\Temp\ipykernel_23856\774128163.py:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('section.PlaceDetailTitle_titleContainer__3sGdf > a').click()


In [ ]:
df['real_date'] = df['date'].apply(change_date)
df = df.rename(columns={'real_date': 'date'})

In [ ]:
df.to_csv("../data/crawling/crawling.csv")
df.to_excel("../data/crawling/crawling.xlsx")

# Data Load

In [2]:
data = pd.read_excel('../data/crawling/crawling.xlsx', index_col=0)
data.head()

,hotel,star,review,date,date.1
0,라까사 호텔 서울,4,접근성 좋고 괝찬은듯.,2일 전,2022. 02. 06
1,라까사 호텔 서울,4,"샤워가운 교체 시급합니다. 허리고리, 주머니가 다 떨어져있고, 실밥이 주렁주렁 달렸...",2022. 01. 30,2022. 01. 30
2,라까사 호텔 서울,5,친절해요:),2022. 01. 14,2022. 01. 14
3,라까사 호텔 서울,1,응대수준이낮음,2022. 01. 09,2022. 01. 09
4,라까사 호텔 서울,1,"최악의 크리스마스 스테이였네요.\n여름에 웨스트 더블룸에서 친구와 호캉스 했었고, ...",2022. 01. 05,2022. 01. 05


### 라까사호텔 서울 언바운디  
- 직원의 불친절 문제, 청소가 구석에 안되어 있는 문제 발생  
- 언바운디의 특별함에 대한 리뷰의 언급은 존재하지 않는 문제점

In [12]:
data.iloc[390:394,:]

,hotel,star,review,date,date.1
390,라까사호텔 서울,1,직원 불친절합니다\n다시는 가고싶지 않아요,2022. 01. 01,2022. 01. 01
391,라까사호텔 서울,5,아주좋았어요,2021. 12. 20,2021. 12. 20
392,라까사호텔 서울,5,이불 너무 푹신하고 좋아요,2021. 10. 04,2021. 10. 04
393,라까사호텔 서울,5,신사올때마다 옵니다 가끔 청소가\n구석엔 안돼있네요ㅠ,2021. 08. 18,2021. 08. 18


### 라까사호텔 광명 언바운디  
- 대체적으로 좋았다는 평이지만, 역시 언바운디의 특별함에 대한 리뷰의 언급은 존재하지 않는 것으로 보이는 문제점

In [10]:
data.iloc[1386:1391,:]

,hotel,star,review,date,date.1
1386,라까사호텔 광명,5,깨끗하고 침대 푹신해요,2021. 11. 22,2021. 11. 22
1387,라까사호텔 광명,5,좋아요 ㅎ,2021. 11. 04,2021. 11. 04
1388,라까사호텔 광명,5,최고였어요.,2021. 10. 20,2021. 10. 20
1389,라까사호텔 광명,5,"엄청 깨끗하고, 직원 분들도 친절해서 행복한 여행이\n되었어요. 주차 공간도 넓어서...",2021. 10. 13,2021. 10. 13
1390,라까사호텔 광명,5,잘 쉬고 잘 묵고 가요~ 원목느낌 디자인도 참 깔끔하고 좋았습니다!,2021. 10. 05,2021. 10. 05


In [21]:
data.iloc[1389].review

'엄청 깨끗하고, 직원 분들도 친절해서 행복한 여행이\n되었어요. 주차 공간도 넓어서 초보운전인데도 불구하고 편하게 주차했습니다 :) 조식도 맛있어요!'

### 코리아나 호텔 언바운디  
- 호텔의 위치적 이점, 직원의 친절함  
- 리모델링을 통해 외관만 깔끔하다는 단점, 시설의 제한적 이용에 대한 단점  
- 언바운디의 특별함에 대한 리뷰의 언급은 역시 존재하지 않는 것으로 보이는 문제점

In [13]:
data.iloc[1877:1879,:]

,hotel,star,review,date,date.1
1877,호텔 코리아나,5,서울 중심인 광화문에 있는 호텔이라 여기저기 구경할 곳은 많아서 좋았습니다 먹을 곳...,2021. 11. 21,2021. 11. 21
1878,호텔 코리아나,2,일단 화장실이 진짜 노후됐고 겉에만 리모델링 잘해놨네요\n헬스장은 일욜에 휴관이라 ...,2021. 10. 31,2021. 10. 31


In [23]:
data.iloc[1877].review

'서울 중심인 광화문에 있는 호텔이라 여기저기 구경할 곳은 많아서 좋았습니다 먹을 곳도 많았구요 ㅎㅎ 살짝 아쉬운 점은 리뉴얼 했다지만 그래도 느껴지는 고전적인 카페트가 좀 아쉬웠던 거 같습니다 그래도 친절하고 다른 점들도 좋아서 재방문하고 싶어요'

In [24]:
data.iloc[1878].review

'일단 화장실이 진짜 노후됐고 겉에만 리모델링 잘해놨네요\n헬스장은 일욜에 휴관이라 이용도 못했고 토욜은 7시에 닫아서 구경도 못했어요\n이용시간을 고지해주셨으면 여기호텔까지 굳이 안왔을것같아요\n티비도 90년대에 산것처럼 옛날티비라 화질도 안좋고 창문도 없어서 환풍구 열어서 환기했어요'

### 나인브릭 호텔 언바운디  
- 기존의 객실의 청결함과 여러가지 기능에 대한 장점에 대한 언급  
- 언바운디에 대한 특별함에 대한 언급이 존재하는 리뷰  
- 리뷰의 갯수가 너무 적다는 단점

In [15]:
data.iloc[3784:3785,:]

,hotel,star,review,date,date.1
3784,호텔 나인브릭,5,체크인할때 선물로 가방패키지 주는거 너무 감동이였구요! 또 객실 컨디션이랑 룸서비스...,2022. 01. 30,2022. 01. 30


In [25]:
data.iloc[3784].review

'체크인할때 선물로 가방패키지 주는거 너무 감동이였구요! 또 객실 컨디션이랑 룸서비스 기능 있는거 못써보긴했지만 다음에 가면 꼭 써보고싶을 정도로 잘되어있어요 다시 방문하고싶을 호텔이예요!'

### 더 리버사이드 호텔 언바운디  
- 객실의 청소상태의 편차가 심한 것으로 판단  
- 언바운디의 특별함에 대한 리뷰는 존재하지 않는 것으로 보이는 문제점

In [16]:
data.iloc[4525:4528,:]

,hotel,star,review,date,date.1
4525,호텔 더 리버사이드,5,아주좋아요,2021. 12. 31,2021. 12. 31
4526,호텔 더 리버사이드,2,방이...충격...주차장도..충격..\n뷔페는 안가봐서모르겠지만 숙소로는 비추에요ㅠ...,2021. 11. 19,2021. 11. 19
4527,호텔 더 리버사이드,5,너무 잘 쉬다 갔습니다. 청결부터 뷰까지 완벽했습니다~,2021. 08. 21,2021. 08. 21


In [26]:
data.iloc[4526].review

'방이...충격...주차장도..충격..\n뷔페는 안가봐서모르겠지만 숙소로는 비추에요ㅠㅠ 정말작은방에 싱글두개 붙여서 킹사이즈 하나인것처럼 합쳐두었고 드라이기 상태도 중고로도 안팔거같은거에 화장실 냄새는..하..ㅠㅠ모텔보다 못한 호텔이었네요..\n그리고 무엇보다 방음이 진짜 최악이었습니다.'

### 평창 고려궁 한옥호텔 언바운디  
- 언바운디만 제공, 기존의 호텔이 존재하지 않기에 비교불가  
- 좋았다는 평들도 있지만, 가격대에 비해 만족하지 못했다는 평가가 존재  
- 언바운디의 패키지에 대한 특별한 리뷰는 존재하지 않음

In [37]:
data.iloc[4528:4538,:]

,hotel,star,review,date,date.1
4528,평창 고려궁 한옥호텔,5,천상의 숙박경험.\n여타 한옥호텔들은 겉만 기와지붕이고 안은 허접한데 여긴 정말 정...,2020. 06. 23,2020. 06. 23
4529,평창 고려궁 한옥호텔,4,친절하고 시설 깔끔하고 좋았습니다\n다만 한파때 갔더니 방이 많이 추웠네요,2021. 12. 30,2021. 12. 30
4530,평창 고려궁 한옥호텔,4,다 좋은데..한복이 입을게 많지않아 아쉬웠고..그리고 퇴실도 안했는데 문압에 청소용...,2021. 08. 20,2021. 08. 20
4531,평창 고려궁 한옥호텔,5,무더운 여름에 정말 힐링이 되는 곳이네요.,2021. 08. 01,2021. 08. 01
4532,평창 고려궁 한옥호텔,5,정말 너무 좋았어요\n직원분도 친절하시고 조식도 정갈하고\n다음에는 몇일 날잡고 아...,2021. 07. 11,2021. 07. 11
4533,평창 고려궁 한옥호텔,3,너무 기대를 했던건지.... 한옥보다는 한옥식 원룸느낌이었습니다. 숲속에 있어서 벌...,2021. 06. 29,2021. 06. 29
4534,평창 고려궁 한옥호텔,5,엄청 깔끔하고 고객응대 대 만족👍🏻,2021. 04. 27,2021. 04. 27
4535,평창 고려궁 한옥호텔,5,눈이 와서 더욱 아름다웠던 고려궁. 조식도 맛있습니다.,2020. 12. 13,2020. 12. 13
4536,평창 고려궁 한옥호텔,5,한옥의 아름다움을 느낄 수있었어요\n저희가 머물렀던 방에 에어컨이 없었는데\n산...,2020. 08. 27,2020. 08. 27
4537,평창 고려궁 한옥호텔,5,여긴 정말 최고 인정,2020. 08. 02,2020. 08. 02


In [28]:
data.iloc[4528].review

'천상의 숙박경험.\n여타 한옥호텔들은 겉만 기와지붕이고 안은 허접한데 여긴 정말 정통 한옥입니다.\n여름에 에어컨 안 틀어도 시원하고, 경치도 너무 좋은데다 방 안에 히노끼 욕조까지 너무 힐링이에요.\n어메니티는 ARDER가 제공됩니다.\n더 놀라운건 이 가격에 조식까지 다 포함이라는 거고, 조식은 정갈한 약선식에 카페테리아에는 멋진 진공관 스피커까지 있어요. 여기야말로 평창의 명소입니다. 강추해요더보기'

In [30]:
data.iloc[4530].review

'다 좋은데..한복이 입을게 많지않아 아쉬웠고..그리고 퇴실도 안했는데 문압에 청소용구 갖다놓는  매너는 좀 아닌듯..'

In [32]:
data.iloc[4532].review

'정말 너무 좋았어요\n직원분도 친절하시고 조식도 정갈하고\n다음에는 몇일 날잡고 아예 폭 박혀있으러 가려구요!!'

In [33]:
data.iloc[4533].review

'너무 기대를 했던건지.... 한옥보다는 한옥식 원룸느낌이었습니다. 숲속에 있어서 벌레야 어쩔 수 없지만 화장실에서 왕개미를 계속 잡았고, 내부에도 먼지가 많네요.🥲 애들 2명식사 및 침구 추가로 5만원이나 현장에서 더 결재했는데, 아침식사도 너무나 평범한 한식이었습니다. 한번 가보았으니, 이젠 안갈거같습니다;;;; 그래도 직원분들 모두 친절했습니다'

In [36]:
data.iloc[4536].review

'한옥의 아름다움을 느낄 수있었어요\n저희가  머물렀던 방에  에어컨이 없었는데\n산바람이 시원해서  선풍기 에어컨이 필요가 없겠더라구요\n바디워시랑욕실화가 있었으면-.....ㅠ\n그부분이 아쉽긴하지만  곳곳에 볼거리가많았어요더보기'

### 광안리 스튜디오 202 호텔 언바운디  
- 위치적이점, 직원의 친절함, 청결도, 여러가지로 만족도가 상당히 높은 호텔  
- 언바운디에 대한 평은 상당히 좋은 것으로 나타남  
- 대부분이 큰 기대없이 방문했다가 생각보다 너무 만족하는 반응  
- 따라서 기본적으로 숙소에 대한 만족도가 높을 때, 언바운디의 패키지가 더 빛날 수 있다는 점을 느낌  
- 조심스럽게 예상해본 결과, 부산에서 위치적 이점을 가지고 좋은 호텔들은 비싸기 때문에 언바운디의 패키지가 들어간 가격이 비싸지 않은 것처럼 느껴졌을 수도 있다고 생각  
- 근본적으로 가격이 싸고, 소비자들이 만족할만한 호텔에 패키지를 제공할 수 있다면 성공요인이 될 것이라고 판단

In [18]:
data.iloc[4545:4587,:]

,hotel,star,review,date,date.1
4545,광안리 스튜디오202,5,저번주에 다녀왔는데 후기 이제야 쓰네용ㅎㅎ🏡\n너무 만족스러웠고 재방문의사 1000...,2021. 11. 24,2021. 11. 24
4546,광안리 스튜디오202,5,업체에서 찍은 사진과 똑같았어요! 깔끔하고 정리정돈 잘되어있어서 사용시 만족했습니다...,2021. 09. 12,2021. 09. 12
4547,광안리 스튜디오202,5,숙소 위치는 광안리 해수욕장 걸어서 1분 거리에 있습니다.\n청결도는 신축 건물이라...,2021. 08. 16,2021. 08. 16
4548,광안리 스튜디오202,5,너무 좋앗어요! ㅎㅎ 뷰가 아쉬웟는대 연박하고싶응 숙소예요!,3시간 전,2022. 02. 08
4549,광안리 스튜디오202,5,너무 이쁘고 깔끔해요\n제일 좋은게 안에서 취사 가능하고 와인잔 등 그릇도 구비되어...,17시간 전,2022. 02. 08
4550,광안리 스튜디오202,5,괜찮았습니다,5일 전,2022. 02. 05
4551,광안리 스튜디오202,5,너무좋았어요+,2022. 02. 01,2022. 02. 01
4552,광안리 스튜디오202,5,너무너무 좋았습니다. 수건도 넉넉했고 3명이라 침구도 추가해주셨습니다. 와인잔과 오...,2022. 01. 26,2022. 01. 26
4553,광안리 스튜디오202,5,잘놀다갑니다,2022. 01. 24,2022. 01. 24
4554,광안리 스튜디오202,4,"숙소의 첫인상은 사진보다 훨씬 예쁘고 깔끔하고 좋았어요. 바다가 안보일줄알았는데, ...",2022. 01. 17,2022. 01. 17


In [38]:
data.iloc[4545,:].review

'저번주에 다녀왔는데 후기 이제야 쓰네용ㅎㅎ🏡\n너무 만족스러웠고 재방문의사 1000%에요!!ㅎㅎ\n\n돈 더 주고 언바운디A타입 객실💒로 묵었습니당~\n코로나라 비대면 입실이었는데 미리 문자💌로 상세히 다 안내 주셔서 너무 편했어용!!\n입실 하자마자 와~ 감탄했어요!!ㅎㅎ \n책상위에 향수 크림 로션 등 뷰티템💄이랑 가방까지 한가득 주셨는데 이렇게나 많이 주실줄은 몰라서 완전 기분 좋았어요..!!ㅋㅋ\n짐 풀고 둘러보면서 사진📸을 요렇코롬 많이 찍었던 곳은 여기가 유일할 것 같아요!!ㅎㅎ\n\n1,2층 전부 다 무드등🪔이 있어서 불을 다 꺼놔도 너무 예뻤어용!!\n숙소가 아담한데 있을 건 다 있어서 너무 좋았습니당ㅎㅎ\n수건도 완전 넉넉히 주셨고, 비품🧼 위치도 하나하나 다 적혀있고, 세탁기랑 칼이나 그릇 와인잔 등 식기류✂️도 널널해서 저녁에 맛있는 거 해 먹으니 좋더라구요~!!ㅎ\n\n온수조절도 너무 잘 되고 에어컨도 잘 되요 완전좋아!!!\n침대근처엔 선풍기도 있었어서 엄청 편하더라고요ㅎㅎ\n📺티비는 넷플릭스랑 유튜브도 되서 노래 틀어두고 저녁에 밥 해먹었는데 그야말로 파티룸..ㅎㅎㅎ\n의자 옆에 통창이라 🪟바깥뷰도 이뻐요!! 해 뜨고질때마다 무드등 켤 때마다 숙소 분위기가 달라지는게 묘한 매력이에요ㅎㅎ\n정말로 숙소 나가서 모퉁이만 돌면 바로 광안리 해변🏝이라 신나게 놀면서 밤 늦게 들어와도 괜찮더라고요ㅎㅎㅎ\n광안리 숙소중에 제일 베스트 인 것 같아요!!ㅋㅋ\n\n어쩌다보니 연박 하게됐는데 늦은 시간까지도 연락 주시고 너무 감사했어요!!\n연박해도 다른 숙소들보다 저렴 하다는 게 포인트... 여기서 살고싶어요ㅠㅠ 너무 이쁘잖아요..!!\n1층 프론트 직원분들도 너무 친절하셨고 체크아웃 시에 귀여운 서비스😊도 주셔서 완전 행복했어요ㅎㅎ\n\n가격도 저렴한데 너무 좋아서 다음에도 또 가겠습니댜ㅎㅎ 호캉스 즐겁게 즐기다 가요!!ㅎㅎ\n집만큼 편하게 지냈어서 나가는 게 아쉽더라고요ㅎㅎ\n다들 고민하지 마시고 꼭 예약하세요!! 늦으면 없어요!!ㅋㅋ\n감사합

In [39]:
data.iloc[4546,:].review

'업체에서 찍은 사진과 똑같았어요! 깔끔하고 정리정돈 잘되어있어서 사용시 만족했습니다 :-)\n친구랑 결혼식 가기 전 하루 묵었는데 너무 깔끔해서 만족했습니다. 퇴실 시 프론트 직원분들 너무 친절하셨어요!\n\n엘베가 느린것 빼고는 문제 없는 숙소😆'

In [40]:
data.iloc[4547,:].review

'숙소 위치는 광안리 해수욕장 걸어서 1분 거리에 있습니다.\n청결도는 신축 건물이라 그런지 화장실부터 객실 내 비품 등 모두 청결하였습니다.\n안내 데스크에는 20시까지 교대로 직원 분이 상주해계신듯 합니다. 친절합니다.\n객실 비품은 치약, 칫솔, 드라이기, 수건, 조리도구(칼, 가위, 도마 등), 식기 등이 모두 갖춰져 있습니다. 조미료 관련 X\n객실 내 드럼세탁기가 존재하지만, 세제가 따로 없어 아쉬웠습니다.\n추후 다시 이용할 생각이 있습니다.\n즐겁게 휴가 보내다가 갑니다. 감사합니다.더보기'

In [41]:
data.iloc[4549,:].review

'너무 이쁘고 깔끔해요\n제일 좋은게 안에서 취사 가능하고 와인잔 등 그릇도 구비되어 있는게 최고예요'

In [43]:
data.iloc[4552,:].review

'너무너무 좋았습니다. 수건도 넉넉했고 3명이라 침구도 추가해주셨습니다. 와인잔과 오프너가 구비되어 있고 바로 앞에 와인가게가 있어서 와인 마시기도 좋습니다:) 조금 아쉬운 점은 엘레베이터가 살짝 느리다는 것..? 다음번에도 재방문 하고싶은 숙소입니다:)'

In [44]:
data.iloc[4554,:].review

'숙소의 첫인상은 사진보다 훨씬 예쁘고 깔끔하고 좋았어요. 바다가 안보일줄알았는데, 아침저녁으로 광안대교 보는 재미도 있었고, 또 해수욕장이 1분거리라서 좋았습니다. 다만 아쉬운점은 숙소에 대해서는 복층구조에서 자야하는 2층이 바닥난방이 되지않아 추웠고,그래서 1층의 온도를 마구올려 바닥은 차갑게 공기를 뜨겁게했더니 너무 건조했습니다ㅠㅠ숙소는 난방을 제외하곤 완전 만족!!\n하지만 엘리베이터가 한대인데 매우 느리고 작아요..마지막날은 캐리어들고 10분넘게 기다렸고, 그것도 사람이 꽉차서 한번은 그냥보내고 두번째는 결국..캐리어만 싣어보내고 전 12층부터 걸어내려갔어요..ㅎ\n그리고 주차시 앞에 주차타워에 하는데, 안내해주신바로는 하루에 만원, 2박 2만원인데, 갑자기 2만2천원을 요구하셔서ㅋㅋㅋㅋㅋ카드로한것도 아닌데ㅜ제가 잘못안건줄알고 2만2천원이요?되묻고 영수증같은거 주신거에 확인하니 2만원이라고 되어있는데 22000원달라고 하신거라서 여기 2만원인데요?라고했더니 2만원달라고 말바꾸시더라구요ㅠㅠㅠ(근데 다음날 차뺄때 다른분께도 만천원달라고 하셔서 그분이 만원으로알고있다니깐, 만천원인데 천원빼주신다며 만원보내라고 말바꾸시더라구요,,)\n요부분은 한번 확인해주셔야할것같아요.\n저는 숙소는 너무 만족해서 재방문의사있어요.\n주변 편의시설도 너무가깝고 좋았어요.더보기'

In [45]:
data.iloc[4559,:].review

'지친 마음 풀려고 나홀로 여행을 왔는데 후기보다 더 좋네요. 일단 숙소가 복층이라 층고도 높고 무드등도 있고 쇼파도 있고, 창도 커서 밤 야경 구경하기 좋았어요. 취사는 안 했지만 취사도 가능하고, 티비도 넷플이랑 유튜브도 볼 수 있어서 좋았습니다. 광안리 해변은 살짝 보이는데 그것 나름대로 또 좋았어요 ㅎㅎ\n전 그냥 작은 기프트 팩이겠지 했는데 에코백 한 가득 있어서 놀랐어요. 그래서 나갈 때 이거 다 주시는 거냐고 한 번 더 물어봤어요 ☺\n숙소 주변에 프랜차이즈 치킨집도 있고 카페도 있고 맛집도 있어서 놀거리도 많았습니다. 퇴근하고 바로 내려 오는 거라서 더 일찍 체크인하지 못해서 아쉬웠어요 ㅠㅠ \n다음에 또 부산에 올 일이 있으면 여기 올 생각이에요 😊\n+숙소에 거의 다 있는데 면도기랑 샤워타월은 없어요. 주변에 올리브영 있으니까 거기서 구매하면 됩니다!더보기'

In [46]:
data.iloc[4560,:].review

'사진과 시설이나 상태도 똑같고 너무 깨끗하고 보안도 좋아서 만족스러웠습니다 사장님도 너무 친절하셔서 편하게 놀다갑니당'

In [47]:
data.iloc[4561,:].review

'이가격에 이 퀄리티 실화인가요..\n사장님도 너무 친절하시고 기프트백도 잘 쓸게요\n바로앞에 편의점 그리고 1분거리에 광안리 바다까지..\n방 청결상태도 최고입니다\n지금까지 방문한 부산 숙소중 가장 가성비 좋고\n깔끔하고 예쁜 감성 숙소였어요\n사장님 설명도 친절하게해주시고\n다들 친절하신거같아요 1박2일 잘 쉬다가요\n감사합니다.\n좋은 추억 만들어주셔서\n다음에도 부산 놀러온다면 재방문할게요 꼭\n감사합니다!더보기'

In [48]:
data.iloc[4562,:].review

'너무 만족스러웠습니다~ 오피스텔 기반이라 그런지 복층 구조도 좋았고 깔끔하고 수납공간도 많아서 넓다고 느껴졌어요'

In [49]:
data.iloc[4564,:].review

'위치 및 청결은 좋았습니다. 하나 아쉬운 건 연박 하면 청소 요구할때 돈을 요청하는 것, 스스로 쓰레기를 치워야 한다는 것..?'

In [50]:
data.iloc[4565,:].review

'광안리 해수욕장이랑 너무 너무 가깝고, 숙소도 정말 깔끔했어요 여자 혼자라서 모텔은 가기 꺼려졌는데, 평일이라 저렴한 가격에 좋은 숙소 묵고 갑니다~!!\n단점을 쓰자면 화장실에 슬리퍼가 있으면 좋겠고, 침대에 먼지가 좀 많아요!'

In [51]:
data.iloc[4569,:].review

'방도 너무 예쁘고 구조에 비해 방음도 괜찮고 기분 좋게 잘묵었습니다. 접근성도 좋고 주변에 편의시설이 충분하고 무엇보다 바다가 걸어서 코앞이라 안보이는건 굳이 문제가 안되네요~ 설명도 충분히 해주셔서 불편함이 없었습니다~ 부산갈 때 묵을 곳으로 기억해둬야겠어요!'

In [52]:
data.iloc[4570,:].review

'사진이랑 똑같고, 직원분들도 넘 친절하고\n넘 청결하고 좋아요^~^\n담에  또 오게 만드는  숙소입니다..'

In [53]:
data.iloc[4571,:].review

'프론트에서 안내도 정말 친절하게 해주시고 방도 너무너무깔끔하고 예쁘고 둘이 놀기 딱 좋아용 광안리 바다랑 위치도 가깝고 조용하고 정말 좋았습니당ㅎㅎㅎ 주차타워가 조금 불편하지만 이 부분은 부산에서 어딜가든 감수해야할 사항이기 때문에 괜찮습니다\n숙소는 다음에 또 이용하고 싶어요!!!'

In [54]:
data.iloc[4573,:].review

'주차비용과 음쓰 봉투를 사서 버려야 한다는 점이 아쉽\n숙소 너무 깔끔하고 좋았어요 즐거운 부산여행했어용'

In [55]:
data.iloc[4574,:].review

'모든 것이 다 만족스러웠어요! \n친절하게 서비스 해주셔서 더 기분이 좋았습니다^^\n광안리 바다 코앞이고 어메니티도 다 있고 수건도 넉넉해서 좋았어요 수납공간도 많구요\n다음에 또 방문하고 싶어요'

In [56]:
data.iloc[4579,:].review

'호캉스 했는데 좋았습니다\n직원분들이 친절하시고 이용하는데 만족스러웠습니다\n신축이라 그런지 매우 깔끔했습니다                                                                                                                   \n주변 먹거리도 많아서 호캉스로 즐기기 충분했습니다'

In [57]:
data.iloc[4581,:].review

'잘 쉬다 갑니다..'

### 서귀포 일레인 호텔 언바운디  
- 언바운디만 제공, 비교불가  
- 숙소가 청결하고, 직원이 친절하다는 평이 존재하기는 하나 약간의 편차가 존재하는 것으로 판단  
- 하지만 다소 위치적으로 숙소가 외진 곳에 위치하여 인프라가 아쉽다는 평가  
- 언바운디에 대한 특별함이 드러난 리뷰는 전혀 존재하지 않은 점

In [19]:
data.iloc[4587:4629,:]

,hotel,star,review,date,date.1
4587,서귀포 일레인 호텔,5,숙소 깨끗하고 따뜻해서 좋았습니다,2022. 01. 14,2022. 01. 14
4588,서귀포 일레인 호텔,5,쉬소가 깨끗하고 따뜻해서 좋았습니다,2022. 01. 14,2022. 01. 14
4589,서귀포 일레인 호텔,5,여기 사장님 넘 친절해서 좋아요ㅠㅠ 항상 밝은 얼굴로 인사해주십니다 깔끔해서 전체적...,2022. 01. 12,2022. 01. 12
4590,서귀포 일레인 호텔,1,다좋은데 도로변에있고 배란다가 이중창이아니여서 \n차가지나갈때마다 소음이 엄청커서 ...,2022. 01. 11,2022. 01. 11
4591,서귀포 일레인 호텔,5,방넓고 깔끔하고 침대 편해요\n서귀포에서 가성비로 가장 강추합니다,2022. 01. 07,2022. 01. 07
4592,서귀포 일레인 호텔,5,진짜 안락한 숙소였어요. 가성비 좋고 깔끔하고 매트리스 너무 좋습니다.,2021. 12. 29,2021. 12. 29
4593,서귀포 일레인 호텔,5,깔끔하고 조용하고 친절하고\n무엇하나 빠지지 않는 곳이었어요\n완전 강추 합니다\n...,2021. 12. 20,2021. 12. 20
4594,서귀포 일레인 호텔,2,주변에 아무것도 없어요 \n제주도는 보통 렌트해서 다니니까 뭐 큰 문제는 안되겠지만...,2021. 12. 13,2021. 12. 13
4595,서귀포 일레인 호텔,5,바이크로 여행중이어서 많이 피곤했었는데 푹신한 침대 덕분에 푹 쉴 수 있었습니다. ...,2021. 11. 19,2021. 11. 19
4596,서귀포 일레인 호텔,5,"방음이 잘되는지 너무도 조용하게 쉴수있었고 무엇다보 침대랑 침구가 푹신, 포근해서 ...",2021. 11. 18,2021. 11. 18


In [58]:
data.iloc[4589].review

'여기 사장님 넘 친절해서 좋아요ㅠㅠ 항상 밝은 얼굴로 인사해주십니다 깔끔해서 전체적으로 컨디션이 넘 좋아서 다음에도 연박으로 묶을 예정입니당ㅎㅎ'

In [59]:
data.iloc[4590].review

'다좋은데 도로변에있고 배란다가 이중창이아니여서 \n차가지나갈때마다 소음이 엄청커서 잠을못자요'

In [60]:
data.iloc[4593].review

'깔끔하고 조용하고 친절하고\n무엇하나 빠지지 않는 곳이었어요\n완전 강추 합니다\n실내도 넓어서 좋았구요\n1층 로비에서 커피 한잔 내려서\n올라와 여유있게 마셨네요\n다음에 또 이용하고 싶은 곳이에요더보기'

In [61]:
data.iloc[4594].review

'주변에 아무것도 없어요 \n제주도는 보통 렌트해서 다니니까 뭐 큰 문제는 안되겠지만 \n차세우고 술이라고 한잔..하며 식사를 하려고하는데 식당 옆에 딱하나 편의점도 500미터 걸어가야하는데 중요한건 인도가 없어서 차도 갓길로 위태롭게 목숨걸고 걸어가야합니다 \n워낙 인적이 없는 곳이라 차쌩쌩 다녀서 아주 위험했어요 \n가시는 분들은 부디 식사 다 하시고 술도 드실 거면 미리 사시고 \n숙소 안에서 잠만 자면 될것 같아요더보기'

In [62]:
data.iloc[4595].review

'바이크로 여행중이어서 많이 피곤했었는데 푹신한 침대 덕분에 푹 쉴 수 있었습니다. 이 정도 금액에 이런 컨디션 쉽지 않은데 좋네요 ^^'

In [63]:
data.iloc[4596].review

'방음이 잘되는지 너무도 조용하게 쉴수있었고 무엇다보 침대랑 침구가 푹신, 포근해서 잠이 솔솔잘왔어요! 방도 따뜻해서 편하게 잘 쉬다갑니다~'

In [64]:
data.iloc[4598].review

'진짜 청결하고 집같은느낌? 방도 크구 침구도 깨끗했어요 진짜 집에서 자는거 같았어여 ❤️❤️넘 좋아서 연박했어요!'

In [65]:
data.iloc[4599].review

'숙소 깔끔하고 너무 젛았어요ㅠ 우드톤에 들어가자마자 따뜻한 공기가 반겨주네요\n사장님이 조용한 방으로 주신다고 하셔서 넘 만족\n방음 정말 잘되더라구요 잘때 편하기 잤답니다 추천해요!'

In [66]:
data.iloc[4600].review

'진짜 너무 깨끗 하구 가성비 진짜 좋아요..!!!!\n그래서 연박했어요 👍🏻 중문쪽 여기저기 다 가깝고 좋습니다.'

In [67]:
data.iloc[4606].review

'너무 조용하고 깨끗한 숙소였습니다^^\n서핑하러 제주에 자주 가는데~ 너무 편하게 잘 쉬다 갑니다 \n다음번에 또 가고 싶습니다 ~^^'

In [68]:
data.iloc[4615].review

'사진으로 본 것보다 훨씬 좋네요ㅎㅎ 훨씬 넓고 깨끗했어요. 제가 가본 호텔 중 제일 넓은 것 같네요! 주변에 뭐가 없는 건 좀 아쉽지만요'

In [69]:
data.iloc[4616].review

'프론트 아줌마 직원 최악으로 불친절해요 진짜 싫었어요 객실 청소 안 돼서 먼지도 은근 많았고 생수 한 병 없어요 그래도 가성비 괜찮은 곳이긴 합니다'

In [70]:
data.iloc[4619].review

'일단 가격대비 방이 깔끔하니 좋았어요\n까사미아가구로 되어잇다하여 기대하고 갔는데 내집같은 느낌! 저희방이 도로변쪽이라 방음이좀 잘안됐어요\n뷰는 사실모딱히 없지만 방도넓고 만족\n다만 차가 없으시다면 조금은 불편할듯해요\n주위에 모가 딱히 잇진 않아서 차를 이용해야합니다더보기'

In [71]:
data.iloc[4621].review

'친절하시고 조용해서 좋았어요. 아이가 있어 예약한 방보다 침대가 큰 곳으로 바꿔주셨어요. 감사했어요^^  차는 타고 주변 다녀야 하구요. 멀지 않아서 저흰 크게 불편하진 않았어요~'

In [72]:
data.iloc[4627].review

'전에 한번 이용했던곳이라 믿고 다시 재방문했어요\n주변에 편의점이 없는게 단점 나머진 다 좋아요\n먹거리는 미리 준비해서 들어가세요^^\n시설도 깨끗하고 방도 넓어서 쾌적해요\n직원분들도 친절하셔서 따뜻하게 하루 머물다\n갑니다더보기'